## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [31]:
import os
import string
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [58]:
folds = ['Friends - season 1', 'Friends - season 2', 'Friends - season 3',
         'Friends - season 4', 'Friends - season 5', 'Friends - season 6',
         'Friends - season 7']

txts = []

dir_line = '/Users/macbook/Downloads/Friends/'

for fold in folds:
    local_dir = list(os.walk(dir_line + fold))
    local_path = str(local_dir[0][0])
    for text in local_dir[0][2]:
        if '.txt' in text and type(text) == str:
            with open(local_path + '/' + text) as file:
                txts.append(cleaning(file.read()))

In [57]:
def cleaning(text):
    tokens = word_tokenize(text.lower())
    for el in tokens:
        if el in string.punctuation:
            tokens.remove(el)
        if el == '...' or el == "''" or el == '—' or el == '«' or el == '»':
            tokens.remove(el)
    text = ' '.join([i for i in tokens if not i.isdigit()])
    return(text)

In [65]:
vec = CountVectorizer()
term_doc_matrix = vec.fit_transform(txts).toarray()
term_doc_matrix_DataFrame = pd.DataFrame(term_doc_matrix,
                                         columns=vec.get_feature_names())

0       1
1       0
2       4
3       3
4       0
5       0
6       3
7       2
8       3
9       5
10      6
11      2
12      2
13      1
14      1
15      5
16      7
17      2
18      1
19      0
20      5
21      2
22      2
23      6
24      3
25      1
26      1
27      1
28      0
29      4
       ..
135     0
136     3
137     6
138     3
139     2
140     5
141     5
142     7
143     2
144     9
145     2
146     8
147     2
148     9
149     7
150    12
151    10
152     4
153     5
154     3
155     3
156     1
157     0
158     4
159     3
160     3
161     6
162     3
163     1
164     3
Name: фиби, Length: 165, dtype: int64

In [91]:
term_doc_matrix_DataFrame.head(n=10)

,00,000,007,009,02,0fps,10,100,11,11б,...,ёкнуло,ёлками,ёлке,ёлки,ёлку,ёлочных,ёпэрэсотэ,ёрл,ёрш,ёще
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
term_doc_matrix_DataFrame.shape

(165, 30829)

In [83]:
text = 'фиби'

term_doc_matrix_DataFrame[term_doc_matrix_DataFrame[text] != 0].index.tolist()

[0,
 2,
 3,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 29,
 31,
 32,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 53,
 54,
 55,
 56,
 58,
 60,
 61,
 62,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 83,
 85,
 86,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 158,
 159,
 160,
 161,
 162,
 163,
 164]

In [90]:
### напишите функцию булева поиска по построенной матрице
def boolean_search(pattern):
    try:
        return term_doc_matrix_DataFrame[term_doc_matrix_DataFrame[pattern.lower()] != 0].index.tolist()[:5]
    except KeyError:
        return 'No pattern in DataSet'

print(boolean_search('Машина'))
print(boolean_search('Чендлер'))


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]

[4, 10, 17, 35, 39]
[0, 1, 2, 3, 4]


<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [28]:
def inverted_index() -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    return

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [36]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    IDF = log(((N - n + 0.5) / (n + 0.5)))
    first = (k1 + 1) * qf
    second = qf + (k1 * ((1-b) + b * (float(dl)/float(avdl))))
    return IDF * (first / second)

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [ ]:
folds = ['Friends - season 1', 'Friends - season 2', 'Friends - season 3',
         'Friends - season 4', 'Friends - season 5', 'Friends - season 6',
         'Friends - season 7']

documents = d

dir_line = '/Users/macbook/Downloads/Friends/'

for fold in folds:
    local_dir = list(os.walk(dir_line + fold))
    local_path = str(local_dir[0][0])
    for text in local_dir[0][2]:
        if '.txt' in text and type(text) == str:
            with open(local_path + '/' + text) as file:
                txts.append(file.read())

In [38]:
def compute_sim() -> float:
    
    return 


def get_search_result() -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    return 